In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.linear_model import LinearRegression
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.tools.tools import add_constant
# import statsmodels.api as sm
# from scipy import stats
# from patsy import dmatrices

# import matplotlib.pyplot as plt

# Problem współliniowości

### Diagnostyka

1. **Macierz korelacji predyktorów** - $D_X = (\rho_{ij})$;

2. **Uwarunkowanie macierzy** $\frac{\lambda_{\text{max}}(D_X)}{\lambda_{\text{min}}(D_X)}$ - duże $\implies$ istnieje para predyktorów zależnych liniowo;

3. **VIF** (ang. *variance inflation factor*) - współczynnik podbicia wariancji

    Dla $1\leq i \leq p-1$: $$R^2_i = \frac{\text{RSS}}{\text{TSS}}$$ dla modelu $x_i \sim x_{-i}$, gdzie $x_{-i}$ oznacza wszystkie zmienne objaśniające z  pominięciem $i$-tej.

    Wówczas
    $$
    \text{VIF}_i = \frac{1}{1-R_i^2}
    $$

    **Interpretacja:** Duża wartość dla pewnego $i$ wskazuje na potencjalną liniową zależność $i$-tej zmiennej objaśniającej od pozostałych zmiennych.

    **Reguła kciuka:** Jeśli $\text{VIF}_i\geq 10$, to $i$-tą zmienną uznajemy w przybliżeniu liniowo zależną od pozostałych.

# Zadanie 1
Dla danych `Carseats` sprawdź, czy występuje w nich problem współliniowości przy użyciu powyższych metod. Jeśli tak, odrzuć ze zbioru zmienne zależne liniowo i dopasuj model regresji liniowej bez nich. Porównaj wyniki.

In [ ]:
# carseats = sm.datasets.get_rdataset(dataname="Carseats", package="ISLR", cache=True)
# carseats_df = carseats.data
# carseats_df = pd.get_dummies(carseats_df,columns = ['ShelveLoc','Urban','US'],drop_first=True)

In [ ]:
# #X ma być bez Sales
# Xvar = carseats_df.loc[:, ~carseats_df.columns.isin(['Sales'])]
# Xvar

In [ ]:
# features = Xvar.columns[np.arange(0,7)] #tylko ciągłe
# correlation_matrix = Xvar[features].corr()
# print(correlation_matrix)

In [ ]:
# #zrobimy heatmapę od wartości bezwzględnych 0 - brak koralcji,  1-mocna korelacja
# import matplotlib as mpl
# heatmap = plt.pcolor(np.abs(correlation_matrix), cmap=mpl.cm.coolwarm, alpha=0.8)

# heatmap.axes.set_frame_on(False)
# heatmap.axes.set_yticks(np.arange(correlation_matrix.shape[0]) + 0.5, minor=False)
# heatmap.axes.set_xticks(np.arange(correlation_matrix.shape[1]) + 0.5, minor=False)
# heatmap.axes.set_xticklabels(features, minor=False)
# plt.xticks(rotation=90)
# heatmap.axes.set_yticklabels(features, minor=False)
# plt.tick_params(axis='both', which='both', bottom='off',
#                     top='off', left='off', right='off')
# plt.colorbar()
# plt.show()

In [ ]:
# y = carseats.data['Sales']
# lm = LinearRegression()
# lm.fit(Xvar.values,y)
# lm.coef_

In [ ]:
# Xvif = add_constant(np.array(Xvar.values,dtype=float))
# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(Xvif, i) for i in range(1,Xvif.shape[1])]
# vif["features"] = Xvar.columns
# vif

In [ ]:
#jest ok, nie trzeba nic wyrzucać

# Zadanie 2
Wczytaj dane `kc_house_data.csv` ([This dataset contains house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015](https://www.kaggle.com/harlfoxem/housesalesprediction/data)).

Dopasuj model `price ~ bathrooms + sqft_living + sqft_lot + sqft_above + sqft_basement + lat + long`, uwzględnij współliniowość predyktorów.

In [ ]:
# house = pd.read_csv('kc_house_data.csv')
# house.head()

In [ ]:
# columns = np.array(['bathrooms', 'sqft_living', 'sqft_lot',
#                     'sqft_above', 'sqft_basement', 'lat', 'long'])
# Xvar = house.loc[:, house.columns.isin(columns)]
# y = house['price']

In [ ]:
# #heatmapa od wartości bezwzględnych korelacji - im bliżej 1 tym większe skorelowanie
# correlation_matrix = house[columns].corr()
# heatmap = plt.pcolor(np.abs(correlation_matrix),alpha =0.8,cmap=mpl.cm.coolwarm)
# heatmap.axes.set_frame_on(False)
# heatmap.axes.set_yticks(np.arange(correlation_matrix.shape[0]) + 0.5, minor=False)
# heatmap.axes.set_xticks(np.arange(correlation_matrix.shape[1]) + 0.5, minor=False)
# heatmap.axes.set_xticklabels(columns, minor=False)
# plt.xticks(rotation=90)
# heatmap.axes.set_yticklabels(columns, minor=False)
# plt.tick_params(axis='both', which='both', bottom='off',
#                     top='off', left='off', right='off')
# plt.colorbar()
# plt.show()


# Zadanie 3
Wczytaj zbiór `Hald.csv`. Znajdź najlepszy model regresji liniowej uwzględniając współliniowość predyktorów.

Opis zbioru:

    Heat evolved during setting of 13 cement mixtures of four basic ingredients. Each ingredient percentage appears to be rounded down to a full integer. The sum of the four mixture percentages varies from a maximum of 99% to a minimum of 95%. If all four regressor X-variables always summed to 100%, the centered X-matrix would then be of rank only 3. Thus, the regression of heat on four X-percentages is ill-conditioned, with an approximate rank deficiency of MCAL = 1. The first column is the response and the remaining four columns are the predictors.

In [ ]:
# hald = pd.read_csv("Hald.csv")
# hald.head()

In [ ]:
# Xvar = hald.loc[:, hald.columns!='y']
# y = hald['y']